In [1]:
import pandas as pd

# Daycare injuries

This notebook joins and exports DFPS datasets so data warehouse users can review daycare violations for potential injuries and other characteristics. It uses data exported Feb 1, 2018.

In [2]:
non_compliance = pd.read_csv('../csv/dfps-2018-02-01/non_compliance.csv').drop('Unnamed: 0', axis=1)
operations = pd.read_csv('../csv/dfps-2018-02-01/operations.csv').drop('Unnamed: 0', axis=1)
inspections = pd.read_csv('../csv/dfps-2018-02-01/assessment.csv').drop('Unnamed: 0', axis=1)

In [3]:
import re
# Get potential injury violations

keywords = [
    'hurt',
    'bruis',
    'injur',
    'fractur',
    'conscious',
    'lacerat'
]

keywords_high = [
    'burn(?! cream)',
    'sprain',
    'concuss',
    'seizure',
    'vomit',
    'tooth (?!brush|paste)',
    'teeth ',
    'responsiv',
]

POTENTIAL_INJURY_KEYWORDS = '|'.join(keywords)

POTENTIAL_INJURY_KEYWORDS_HIGH = '|'.join(keywords_high)

potential_injury_violations = pd.concat([
    non_compliance[
        non_compliance.STANDARD_NUMBER_DESCRIPTION.str.contains(
            'injur',
            regex=True,
            flags=re.IGNORECASE
        ) |
        non_compliance.NARRATIVE.str.contains(
            POTENTIAL_INJURY_KEYWORDS,
            regex=True,
            flags=re.IGNORECASE
        )
    ],
    non_compliance[
        non_compliance.NARRATIVE.fillna('').str.contains(
            POTENTIAL_INJURY_KEYWORDS_HIGH,
            regex=True,
            flags=re.IGNORECASE
        )
    ].query(
        'STANDARD_RISK_LEVEL == "High" | STANDARD_RISK_LEVEL == "Medium High"'
    ),
    non_compliance[
        non_compliance.NARRATIVE.astype(str).str.contains(
            'brok|break(?!fast)',
            regex=True,
            flags=re.IGNORECASE
        )
    ].query('STANDARD_RISK_LEVEL == "High"')
])

In [4]:
# Get all violations from inspections
# where a potential injury occurred.

potential_injury_violations_and_related = non_compliance[
    non_compliance.ACTIVITY_ID.isin(
        potential_injury_violations.ACTIVITY_ID.unique()
    )
]

In [5]:
export = potential_injury_violations_and_related.merge(
    operations,
    on='OPERATION_ID'
).merge(
    inspections.drop(
        'OPERATION_ID',
        axis=1
    ),
    on='ACTIVITY_ID'
).assign(
    # add a column to signify if violation has a potential injury
    potential_injury = lambda x: x.STANDARD_NUMBER_DESCRIPTION.str.contains(
        'injur',
        regex=True,
        flags=re.IGNORECASE
    ) | x.NARRATIVE.str.contains(
        POTENTIAL_INJURY_KEYWORDS,
        regex=True,
        flags=re.IGNORECASE
    ) | x.NARRATIVE.str.contains(
        POTENTIAL_INJURY_KEYWORDS_HIGH,
        regex=True,
        flags=re.IGNORECASE
    )
).query(
    'OPERATION_TYPE != "Child Placing Agency" & OPERATION_TYPE != "General Residential Operation"'
)

In [6]:
export.to_csv('../export/export.csv')